# Step 8: Performance Interpretability
---
Goal of the notebook: Implement the permutation importance method and/or
the XPER method to identify the main drivers of the predictive performance of your model.
Are the drivers of the performance metric (Step 8) similar to the drivers of the individual
forecasts identified by SHAP (Step 7). 


Inputs of the notebook:

Output of the notebook:


Takeaways:

Questions, thoughts and remarks:

## Dependencies and path
Adjust the argument in `sys.path.append` to align with your specific requirements.

In [1]:
import pandas as pd 
import numpy as np
import os
import sys

%load_ext autoreload
%autoreload 2

os.chdir('C:/Users/dorab/OneDrive - Ecole Polytechnique/Documents/DSB - HECxX/Year 2/Algorithmic Fairness and Interpretability/afi_final_project')

## 1. Load the data

In [ ]:
df = pd.read_excel("data/dataproject2024.xlsx")

In [ ]:
X = df.drop(["Default (y)", "Pred_default (y_hat)", "ID", "PD", "Group"], axis=1)
y = df["Default (y)"]